In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib


import sys

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/onCRC_WB/BL14_genepromSM_0.7.txt_bestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'


#gtcol='label2'

##### carefully input the model folder#####

modelfol='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/onPBMC/BL14_genepromSM_0.7.txt_REALTRAINbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl_model_10000'


indf=pd.read_pickle(infile)

indf.head()

index  CD4-others  CD8-others  nB-others  \
0  A00354:526:H7N2TDSX2:3:1101:16224:35665    0.770654    0.667090  -0.174346   
1  A00354:526:H7N2TDSX2:3:1101:18249:16532   -0.412692   -0.460795  -0.570282   
2   A00354:526:H7N2TDSX2:3:1101:2889:19304   -0.114256   -0.066513  -0.058256   
3  A00354:526:H7N2TDSX2:3:1101:31882:30123    1.475179    1.424385  -0.371923   
4   A00354:526:H7N2TDSX2:3:1101:5204:35462    1.460526    1.082167  -0.411167   

   NK-others  Mn-others  mNeu-others  m8-others  DC-others  Eo-others  ...  \
0  -0.189423  -0.222449    -0.201987  -0.013346  -0.180090  -0.220654  ...   
1  -0.334795  -0.643872    -0.605462   2.053641  -0.574949  -0.595051  ...   
2  -0.069385   0.009590     0.962667  -0.094872  -0.009077  -0.041026  ...   
3  -0.267462  -0.363308    -0.408897   0.086487  -0.371923  -0.307128  ...   
4   0.009910  -0.378141    -0.445628  -0.322679  -0.387115  -0.244603  ...   

   total_cpg  LENhypoCpG_BY_total_cpg  maxscoredCT_beforeCpGweight  \
0        1.0                      1.0                   CD4-others   
1        3.0                      1.0                    m4-others   
2        1.0                      1.0                  mNeu-others   
3        2.0                      1.0                   CD4-others   
4        2.0                      1.0                   CD4-others   

   maxscore_beforeCpGweight  maxscoredCT  maxscore  deltabasedfragassignment  \
0                  0.770654   CD4-others  0.770654                CD4-others   
1                  2.251256    m4-others  2.251256                 m4-others   
2                  0.962667  mNeu-others  0.962667               mNeu-others   
3                  1.475179   CD4-others  1.475179                CD4-others   
4                  1.460526   CD4-others  1.460526                CD4-others   

                                            filename  deltaMedian  \
0  healthy_normal-NU-35-Bulk-WB_sorted_binnedstat...    -0.163577   
1  healthy_normal-NU-35-Bulk-WB_sorted_binnedstat...    -0.423103   
2  healthy_normal-NU-35-Bulk-WB_sorted_binnedstat...    -0.076923   
3  healthy_normal-NU-35-Bulk-WB_sorted_binnedstat...    -0.333513   
4  healthy_normal-NU-35-Bulk-WB_sorted_binnedstat...    -0.350410   

      deltaMean  
0 -7.137148e-17  
1 -1.427430e-16  
2  2.379049e-17  
3  2.379049e-17  
4 -9.516197e-17  

[5 rows x 27 columns]

In [2]:
features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
def calscore(fgrouped,fname):
    celltype_probadded=defaultdict(list)
    
    celltype_proportiondict=defaultdict(list)
    

    for score in scorecolumns:

        tempct=score.replace('-others','')



        indfsingleCT=fgrouped[fgrouped['deltabasedfragassignment']==score].copy()





    
        testX=indfsingleCT[features]

        RDF_classifier =joblib.load(modelfol+"/"+tempct+".pkl")
        
        
        predictedCT=(RDF_classifier.predict(testX)).tolist()
        predictedTARGETCT=predictedCT.count(1)
        predictedNON_TARGETCT=predictedCT.count(0)
        
        totalpredictedCT=len(predictedCT)
        
        if (predictedTARGETCT+predictedNON_TARGETCT) != totalpredictedCT:
            print ("counting error. Exiting")
            sys.exit(1)
        
        
        cellularproportion=1.0*predictedTARGETCT/totalpredictedCT
        
        
        celltype_proportiondict[tempct]=celltype_proportiondict[tempct]+[cellularproportion]
        

        

        predict_test=RDF_classifier.predict_proba(testX)
        
        tempctprob=(predict_test[:,1]).tolist()
        
        tempct_sumprob=sum(tempctprob)
        
        celltype_probadded[tempct]=celltype_probadded[tempct]+[tempct_sumprob]
        
        
 
        
    
    
    celltype_proportiondictDF=pd.DataFrame.from_dict(celltype_proportiondict)
    celltype_proportiondictDF['Mixture']=fname
    
    celltype_probaddeddf=pd.DataFrame.from_dict(celltype_probadded)
    celltype_probaddeddf['Mixture']=fname
    return celltype_proportiondictDF, celltype_probaddeddf
    

In [5]:
filegrouped=indf.groupby('filename')

probaddedallfilelist=[]
ctproportionallfilelist=[]
for name, group in filegrouped:
    tempctproportion,tempprobadded=calscore(group,name)
    
    ctproportionallfilelist.append(tempctproportion)
    probaddedallfilelist.append(tempprobadded)
    

In [6]:
ctproportionallfilelistDF=pd.concat(ctproportionallfilelist)
ctproportionallfilelistDF.set_index('Mixture',inplace=True)
probaddedallfilelistDF=pd.concat(probaddedallfilelist)
probaddedallfilelistDF.set_index('Mixture',inplace=True)

In [7]:
ctproportionallfilelistDFNORMAL=ctproportionallfilelistDF.div(ctproportionallfilelistDF.sum(axis=1), axis=0)

In [8]:
ctproportionallfilelistDF.to_csv(infile+"_RFctprop_CSxOut.txt",sep="\t")
ctproportionallfilelistDFNORMAL.to_csv(infile+"_RFctpropNORMALIZED_CSxOut.txt",sep="\t")

probaddedallfilelistDF.to_csv(infile+"_RFprob_CSxOut.txt",sep="\t")

In [9]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

1849.1062309741974
